In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mnist-dataset/mnist_dataset.csv
/kaggle/input/mini-heart/Heart.csv


In [2]:
heart_df = pd.read_csv('/kaggle/input/mini-heart/Heart.csv')
heart_df = heart_df.dropna()

# get outputs first 
y = heart_df.AHD.replace("No", 0).replace("Yes", 1)

# begin preprocessing
heart_df = pd.get_dummies(heart_df.drop('AHD', axis=1)).replace(False, 0).replace(True, 1).drop('Unnamed: 0', axis=1)

/tmp/ipykernel_17/2717640149.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = heart_df.AHD.replace("No", 0).replace("Yes", 1)
/tmp/ipykernel_17/2717640149.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  heart_df = pd.get_dummies(heart_df.drop('AHD', axis=1)).replace(False, 0).replace(True, 1).drop('Unnamed: 0', axis=1)


In [3]:
# scaler
from sklearn.preprocessing import MinMaxScaler

# Create the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the data
heart_df_scaled = pd.DataFrame(scaler.fit_transform(heart_df), columns=heart_df.columns)
heart_df_scaled

,Age,Sex,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,ChestPain_asymptomatic,ChestPain_nonanginal,ChestPain_nontypical,ChestPain_typical,Thal_fixed,Thal_normal,Thal_reversable
0,0.708333,1.0,0.481132,0.244292,1.0,1.0,0.603053,0.0,0.370968,1.0,0.000000,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,0.791667,1.0,0.622642,0.365297,0.0,1.0,0.282443,1.0,0.241935,0.5,1.000000,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.791667,1.0,0.245283,0.235160,0.0,1.0,0.442748,1.0,0.419355,0.5,0.666667,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.166667,1.0,0.339623,0.283105,0.0,0.0,0.885496,0.0,0.564516,1.0,0.000000,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.250000,0.0,0.339623,0.178082,0.0,1.0,0.770992,0.0,0.225806,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,0.583333,0.0,0.433962,0.262557,0.0,0.0,0.396947,1.0,0.032258,0.5,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,1.0
293,0.333333,1.0,0.150943,0.315068,0.0,0.0,0.465649,0.0,0.193548,0.5,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,1.0
294,0.812500,1.0,0.471698,0.152968,1.0,0.0,0.534351,0.0,0.548387,0.5,0.666667,1.0,0.0,0.0,0.0,0.0,0.0,1.0
295,0.583333,1.0,0.339623,0.011416,0.0,0.0,0.335878,1.0,0.193548,0.5,0.333333,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [4]:
X = heart_df_scaled
X

,Age,Sex,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,ChestPain_asymptomatic,ChestPain_nonanginal,ChestPain_nontypical,ChestPain_typical,Thal_fixed,Thal_normal,Thal_reversable
0,0.708333,1.0,0.481132,0.244292,1.0,1.0,0.603053,0.0,0.370968,1.0,0.000000,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,0.791667,1.0,0.622642,0.365297,0.0,1.0,0.282443,1.0,0.241935,0.5,1.000000,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.791667,1.0,0.245283,0.235160,0.0,1.0,0.442748,1.0,0.419355,0.5,0.666667,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.166667,1.0,0.339623,0.283105,0.0,0.0,0.885496,0.0,0.564516,1.0,0.000000,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.250000,0.0,0.339623,0.178082,0.0,1.0,0.770992,0.0,0.225806,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,0.583333,0.0,0.433962,0.262557,0.0,0.0,0.396947,1.0,0.032258,0.5,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,1.0
293,0.333333,1.0,0.150943,0.315068,0.0,0.0,0.465649,0.0,0.193548,0.5,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,1.0
294,0.812500,1.0,0.471698,0.152968,1.0,0.0,0.534351,0.0,0.548387,0.5,0.666667,1.0,0.0,0.0,0.0,0.0,0.0,1.0
295,0.583333,1.0,0.339623,0.011416,0.0,0.0,0.335878,1.0,0.193548,0.5,0.333333,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [5]:
# train-test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42 # Setting a random_state ensures reproducibility
)

# 1. manual autodiff implementation
Simple classification on the mini-heart dataset from CS1090b.

## Autodiff primitives
- `x`
- `ax`
- `x + a`
- `1/x`
- `np.exp(x)`
- `np.log(x)`
- `relu(x)`
- `loss`

## Autodiff-based layers
Each layer needs a `forward` (apply primitives) and `backward` (calculate chain rule). Both of these should be vectorised.


## Model architecture
*NOTE: model has no bias rn for simplicity*
1. input
2. lin -> relu (18)
3. lin -> sigmoid predictor

In [6]:
class Autodiff:
    def __init__(self):
        self.operations = []

    def id(self, x):
        return x

    def did(self, x):
        return 1

    def matmul(self, x, W):
        return x @ W

    def dmatmul(self, x, W):
        return W

    def add(self, x, a):
        return a + x

    def dadd(self, x, a):
        return 1

    def rec(self, x):
        return 1 / x

    def drec(self, x):
        return -(1/x)**(2)

    def e(self, x):
        return np.exp(x)

    def de(self, x):
        return np.exp(x)

    def log(self, x):
        return np.log(x)

    def dlog(self, x):
        return 1/x

    def relu(self, x):
        return np.maximum(x, 0)

    def drelu(self, x):
        return (x > 0).astype(x.dtype)

a = Autodiff()

In [7]:
class Linear:
    def __init__(
        self,
        input_size,
        num_nodes
    ):
        self.W = np.random.uniform(-1.0, 1.0, size=(num_nodes, input_size))
        self.b = np.random.uniform(-1.0, 1.0, size=(1, num_nodes))

        # cache
        self.a = None
        self.dprev = None
    

    def forward(self, x):
        return a.add(a.matmul(x, np.transpose(self.W)), self.b)

    def backward(self, dprev):
        self.dprev = dprev
        return dprev * a.dmatmul(self.a, self.W)

class Sigmoid:
    def __init__(
        self,
    ):
        # cache
        self.a = None

    def forward(self, x):
        self.a = a.rec(a.add(a.e(-x), 1))
        return self.a

    def backward(self, dprev):
        return dprev * self.a * (1 - self.a)
        

class ReLU:
    def __init__(
        self
    ):
        # cache
        self.a = None

    def forward(self, x):
        self.a = a.relu(x)
        return self.a

    def backward(self, dprev):
        return dprev * a.drelu(self.a)
        

In [8]:
INPUT_SIZE = X_train.shape[1]

class BasicCLSNet:
    def __init__(
        self,
        input_size
    ):
        self.layers = [
            Linear(
                18,
                18
            ),
            ReLU(),
            Linear(
                18,
                1
            ),
            Sigmoid()
        ]

    def forward(self, x):
        curr_input = x
        for l in self.layers:
            curr_input = l.forward(curr_input)
            # print(f"AFTER LAYER {l}: ", curr_input)

        return curr_input

    def dLdy(self, y, p, loss_grad):
        return loss_grad(y, p)
    
    def backward(self, x,  y, p, loss_grad):
        reverse_layers = self.layers[::-1]
        
        dprev = self.dLdy(y, p, loss_grad)
        grads = [dprev]
        for l in reverse_layers:
            grads.append(dprev*l.backward(dprev))

        return grads
        

cls_model = BasicCLSNet(INPUT_SIZE)

In [9]:
row_in = X_train.iloc[0, :]
y = np.array([y_train.to_numpy()[0]])

def loss(y, p):
    y1 = a.matmul(-y, a.log(p))
    y2 = a.matmul(a.add(1, -y), a.log(a.add(1, -p)))
    return y1 - y2

def loss_grad(y, p):
    y1 = a.matmul(y, p)
    y2 = a.matmul(a.add(1, -y), a.rec(a.add(1, -p)))
    return -y1 + y2

def train_step(x, y, 
               model, 
               loss_grad, 
               lr=1e-3):
    # print("BEGINNING OF STEP")
    # print_weights(model)
    pred = model.forward(x)
    curr_loss = loss(y, pred)
    print("Current loss is: ", curr_loss)

    # calculate gradients
    grads = model.backward(x, y, pred, loss_grad)
    # print(grads)
    # print(grads)
    grads_ordered = grads[::-1]

    # apply to params layer by layer -- only linear (only they have params)
    lin_layers = [i for i in range(len(model.layers)) if isinstance(model.layers[i], Linear)]
    # print(lin_layers)
    
    for idx in lin_layers:
        # print("BEFORE UPDATE: ", model.layers[idx].W)
        model.layers[idx].W -= lr*grads_ordered[idx]
        model.layers[idx].b -= lr*model.layers[idx].dprev
        # print("AFTER UPDATE: ", model.layers[idx].W)
    # print("END OF STEP")
    # print_weights(model)

In [10]:
row_in = X_train.iloc[2, :]
y = np.array([y_train.to_numpy()[2]])

train_step(row_in, y, cls_model, loss_grad)

Current loss is:  [1.59782391]


In [11]:
def print_weights(model):
    for i, l in enumerate(model.layers):
        print(f"----- Layer {i} --- {l} -----")
        print(l.W if isinstance(l, Linear) else "Not a linear layer")

def print_weights_sanity_check(model):
    print(model.layers[2].W)

In [12]:
# for i in range(2):
print("WEIGHTS BEFORE: ")
print_weights_sanity_check(cls_model)
for i in range(X_train.shape[0]):
    # print(f"--- Example {i} ---")
    row_in = X_train.iloc[i, :]
    y = np.array([y_train.to_numpy()[i]])

    # one training step
    train_step(row_in, y, cls_model, loss_grad)

    # print weights for sanity check
    # print_weights(cls_model)

print("WEIGHTS AFTER: ")
print_weights_sanity_check(cls_model)

WEIGHTS BEFORE: 
[[-0.22453583  0.77310758  0.73658135 -0.14661356 -0.62810004 -0.3141743
  -0.27827041  0.06395691  0.13021711 -0.22720213  0.29390156 -0.10687078
   0.20190783  0.30712297  0.49781071  0.54518867 -0.36985091 -0.66500204]]
Current loss is:  [1.22218045]
Current loss is:  [1.59419851]
Current loss is:  [1.50979614]
Current loss is:  [0.10235393]
Current loss is:  [0.74788186]
Current loss is:  [0.69536369]
Current loss is:  [0.43585768]
Current loss is:  [1.59032293]
Current loss is:  [0.84076426]
Current loss is:  [0.47520725]
Current loss is:  [1.24617651]
Current loss is:  [0.10425534]
Current loss is:  [0.90935518]
Current loss is:  [1.0515279]
Current loss is:  [0.61531115]
Current loss is:  [0.40370165]
Current loss is:  [1.21007693]
Current loss is:  [1.0913503]
Current loss is:  [0.78696304]
Current loss is:  [1.52697607]
Current loss is:  [0.28552851]
Current loss is:  [0.80712675]
Current loss is:  [0.3041913]
Current loss is:  [0.47506921]
Current loss is:  [

# 2. simple PyTorch implementation

In [13]:
BATCH_SIZE = 4

X_train_numpy = X_train.to_numpy()
X_train_torch = torch.from_numpy(X_train_numpy).float()
y_train_numpy = y_train.to_numpy()
y_train_torch = torch.from_numpy(y_train_numpy).float()

X_test_numpy = X_test.to_numpy()
X_test_torch = torch.from_numpy(X_test_numpy).float()
y_test_numpy = y_test.to_numpy()
y_test_torch = torch.from_numpy(y_test_numpy).float()

train_ds = TensorDataset(X_train_torch, y_train_torch)
test_ds = TensorDataset(X_test_torch, y_test_torch)
train_dataloader = DataLoader(train_ds, batch_size = BATCH_SIZE)
test_dataloader = DataLoader(test_ds, batch_size = BATCH_SIZE)


In [14]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.cls_block = nn.Sequential(
            nn.Linear(18, 6),
            nn.ReLU(),
            nn.Linear(6, 6),
            nn.ReLU(),
            nn.Linear(6, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        logits = self.cls_block(x)
        return logits

model = NN().to(device)
model

Using cpu device


NN(
  (cls_block): Sequential(
    (0): Linear(in_features=18, out_features=6, bias=True)
    (1): ReLU()
    (2): Linear(in_features=6, out_features=6, bias=True)
    (3): ReLU()
    (4): Linear(in_features=6, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

In [15]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

In [16]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        y = y.unsqueeze(1)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [17]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    print(size)
    print(num_batches)
    
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            y = y.unsqueeze(1)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            
            # check correctness
            result = ((pred < 0.5) == (y < 0.5)).sum().item()
            correct += result

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [18]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.637819  [    4/  237]
60
15
Test Error: 
 Accuracy: 60.0%, Avg loss: 0.679114 

Epoch 2
-------------------------------
loss: 0.637836  [    4/  237]
60
15
Test Error: 
 Accuracy: 60.0%, Avg loss: 0.679084 

Epoch 3
-------------------------------
loss: 0.637852  [    4/  237]
60
15
Test Error: 
 Accuracy: 60.0%, Avg loss: 0.679054 

Epoch 4
-------------------------------
loss: 0.637868  [    4/  237]
60
15
Test Error: 
 Accuracy: 60.0%, Avg loss: 0.679024 

Epoch 5
-------------------------------
loss: 0.637884  [    4/  237]
60
15
Test Error: 
 Accuracy: 60.0%, Avg loss: 0.678994 

Epoch 6
-------------------------------
loss: 0.637900  [    4/  237]
60
15
Test Error: 
 Accuracy: 60.0%, Avg loss: 0.678963 

Epoch 7
-------------------------------
loss: 0.637915  [    4/  237]
60
15
Test Error: 
 Accuracy: 60.0%, Avg loss: 0.678933 

Epoch 8
-------------------------------
loss: 0.637930  [    4/  237]
60
15
Test Error: 
 Accuracy: 60.0